In [134]:
import pandas as pd
import numpy as np

In [163]:
def fpre_process(fname):
    # Hot encoding for all categorical columns
    # Impute values for nulls in categorical and numerical columns

    # 1. Get the selected labels and features - Done
    # 2. Determine the cat / numerical in features - Done
    # 3. Determine columns with Nulls - Done
    # 4. Determine most frequent in Cat - Done
    # 5. Impute cat column values with most frequent - Done
    # 6. Detemine mean / average for Num - Done 
    # 7. Impute num column values with mean - Done
    # 8. Standardize
    # 9. Return X and y

    # for testing - Replace this code with extracting from label_features.py
    label = ['Survived']
    features = ['Pclass', 'Sex', 'Age', 'Cabin']
    
    from base_script_eda import fload_csv
    
    df, size = fload_csv(fname)
    #print df.describe(include = ['O'])

    # calculate the numerical and cat columns

    df_bkp = df[features]
    cols = list(df[features].columns)
    num_cols = list(df[features]._get_numeric_data().columns)
    cat_cols = list(set(cols) - set(num_cols))
    #print cols, num_cols, cat_cols
    
    
    # Determine columns with Null values and Replace NAs
    
    null_num_cols = df[num_cols].columns[df[num_cols].isnull().any()].tolist()
    null_cat_cols = df[cat_cols].columns[df[cat_cols].isnull().any()].tolist()
    print null_num_cols, null_cat_cols
    
    # For numerical columns
    
    for col in null_num_cols:
        print "Before " + str(df[col].isnull().sum())
        df[col].replace(np.NaN, df[col].mean(), inplace=True)
        print "After " + str(df[col].isnull().sum())
    
    # For categorical columns
    
    for col in null_cat_cols:
        print "Before " + str(df[col].isnull().sum())
        df[col].replace(np.NaN, df[col].value_counts().max(), inplace=True)
        print "After " + str(df[col].isnull().sum())
    
        

In [164]:
fpre_process('train.csv')

train.csv successfully loaded
['Age'] ['Cabin']
Before 177
After 0
Before 687
After 0
